### Libraries

In [ ]:
!pip install tensorflow==2.14

In [ ]:
# Fix randomness and hide warnings
seed = 42
input_shape = (96, 96, 3)

import os
os.environ['PYTHONHASHSEED'] = str(seed)

import numpy as np
import math
np.random.seed(seed)
import pandas as pd

# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras.applications.convnext import preprocess_input
from tensorflow.keras import layers as tfkl
from tensorflow.keras import regularizers
from tensorflow.keras import mixed_precision

l2_reg = 0.001

tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# Import other libraries
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold
from sklearn.utils.class_weight import compute_class_weight

mixed_precision.set_global_policy('mixed_float16')

In [ ]:
(tf.__version__, np.__version__)

### Data Loading and class balancing

In [ ]:
data = np.load('../input/plants-clean/clean_data.npz', allow_pickle=True)
imgs, labels_str = data["data"], data["labels"]
labels = (labels_str == "unhealthy").astype("int")

del labels_str, data

# Shuffle original dataset before any operation
indices = np.arange(len(labels))
np.random.shuffle(indices)
imgs = imgs[indices]
labels = labels[indices]

X_train, X_val, y_train, y_val = train_test_split(imgs, labels, test_size=.2)
X_train.shape, y_train.shape

In [ ]:
# We calculate class weights and use them to have a more balanced training process
class_weights = compute_class_weight(
    'balanced',
    classes = np.unique(y_train),
    y = y_train
)

class_weight_dict = dict(enumerate(class_weights))

## ConvNeXt Model

In [ ]:
convnext = tfk.applications.ConvNeXtLarge(
      input_shape = input_shape,
      include_top = False,
      pooling='avg'
  )

convnext.trainable = False
#convnext.summary()

In [ ]:
inputs = tfk.Input(shape=input_shape)
height = width = 96

x = tf.keras.Sequential([
    tfkl.RandomFlip("horizontal"),
    tfkl.RandomTranslation(0.125, 0.125),
    tfkl.RandomRotation(0.12),
    tfkl.RandomZoom(0.03),
    tfkl.RandomFlip("vertical"),
])(inputs)


x = convnext(x)

x = tfkl.Dense(
    75,
    activation='selu',
    kernel_regularizer=tfk.regularizers.l2(1e-5),
    name="c_dense0")(x)
x = tfkl.Dropout(0.3, name="drop0")(x)

outputs = tfkl.Dense(1, activation='sigmoid', name="c_output", kernel_regularizer=tfk.regularizers.l2(1e-5))(x)

model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

early_stopping = tfk.callbacks.EarlyStopping(
    monitor='val_accuracy',
    mode='max',
    patience=15,
    restore_best_weights=True,
    start_from_epoch=15)

reduce_lr_on_plateau = tfk.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.8,
    patience=5
    )

callbacks = [early_stopping, reduce_lr_on_plateau]

In [ ]:
model.compile(
    loss=tfk.losses.BinaryCrossentropy(),
    optimizer=tfk.optimizers.AdamW(),
    metrics=['accuracy']
)
model.summary()

## Training part 1 - transfer learning
The first training is done using transfer learning.
ConvNeXtLarge as feature extractor (non-trainable), followed by one 75 unit dense layer and the final single unit layer.

In [ ]:
history = model.fit(
    X_train,
    y_train,
    class_weight = class_weight_dict,
    validation_data=(X_val, y_val),
    epochs = 85,
    callbacks = callbacks,
    batch_size=128
).history

In [ ]:
# Plot the training
plt.figure(figsize=(15,5))
plt.plot(history['loss'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(history['val_loss'], label='CNN with Augmentation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Binary Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(history['accuracy'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(history['val_accuracy'], label='CNN with Augmentation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

In [ ]:
model.save("/kaggle/working/ConvNeXtLarge_TL_reg")
del model

In [ ]:
import gc
gc.collect()

In [ ]:
!zip -r /kaggle/working/base_cnl_tl_reg.zip /kaggle/working/ConvNeXtLarge_TL_reg

## Training part 2 - Fine tuning
The second part of the training is done by unfreezing the last n layers of ConvNeXt feature extraction, or even making it fully trainable in the last trial we have done (reaching 0.9411 validation accuracy)

In [ ]:
# Reload model
model_convTL = tfk.models.load_model('/kaggle/input/cnl-fu/Model')

In [ ]:
# We both tried using only L2, and L1L2 regularization. Training performances hasn't change significantly.
model_convTL.get_layer("c_dense0").kernel_regularizer = tfk.regularizers.L1L2(1e-2, 1e-2)
model_convTL.get_layer("c_output").kernel_regularizer = tfk.regularizers.L1L2(5e-3, 5e-3)

In [ ]:
model_convTL.get_layer("convnext_large").trainable = True

In [ ]:
def compile_model(model):
    model.compile(
        loss=tfk.losses.BinaryCrossentropy(),
        optimizer=tfk.optimizers.AdamW(learning_rate=1e-5, weight_decay=5e-5),
        metrics=['accuracy']
    )

compile_model(model_convTL)
model_convTL.summary()

In [ ]:
early_stopping = tfk.callbacks.EarlyStopping(
    monitor='val_accuracy',          # Monitor validation loss
    mode='max',                  # Mode 'min' because we want to minimize loss
    patience=15,                  # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True,
)   # Restore model weights from the epoch with the best value of the monitored quantity

reduce_lr_on_plateau = tfk.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.7,
    patience=5
)

callbacks = [early_stopping, reduce_lr_on_plateau]

In [ ]:
# makes only the last N layers of the model trainable
def unfreeze_model_layers(model, N):
    for layer in model.get_layer('convnext_large').layers[:-N]:
        layer.trainable = False
    for layer in model.get_layer('convnext_large').layers[-N:]:
        layer.trainable = True

    print(f"Unlocked: {sum(layer.trainable for layer in model.get_layer('convnext_large').layers)}/{len(model.get_layer('convnext_large').layers)}")
    compile_model(model)

In [ ]:
import gc
gc.collect()

In [ ]:
#unfreeze_model_layers(model_convTL, 180)
history = model_convTL.fit(
    X_train,
    y_train,
    class_weight = class_weight_dict,
    validation_data=(X_val, y_val),
    epochs = 90,
    callbacks = callbacks,
    batch_size=64
).history

64 unfreeze, 5e-4, 1e-4: 0.9031

68 unfreeze, 5e-4, 1e-4: 0.9081

60 unfreeze, 5e-4, 1e-4: 0.9081

80 unfreeze, 5e-4, 1e-4: 0.9091

120 unfreeze, 5e-4, 1e-4: 0.9111

140 unfreeze, 5e-4, 1e-4: 0.9271

200 unfreeze, 5e-4, 1e-4: 0.9281

fully unfeeze, 5e-4, 1e-4: 0.9411

In [ ]:
# Plot the training
plt.figure(figsize=(15,5))
plt.plot(history['loss'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(history['val_loss'], label='CNN with Augmentation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Binary Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(history['accuracy'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(history['val_accuracy'], label='CNN with Augmentation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

In [ ]:
model_convTL.evaluate(X_val, y_val)

In [ ]:
model_convTL.save("/kaggle/working/ConvNeXt_fully_unlocked")

In [ ]:
!zip -r Model_cn_fu.zip /kaggle/working/ConvNeXt_fully_unlocked

In [ ]:
!ls

In [ ]:
from IPython.display import FileLink
FileLink(r'Model_cn_fu.zip')

## Improving validation performances with TEST TIME AUGMENTATION


In [ ]:
import tensorflow.image as tfi
import scipy as sp

def hor_shift(images):
    return np.roll(images, 15, axis=2)
def rot90(images, k):
    return tfi.rot90(images,k)
def flip_lr(images):
    return tfi.flip_left_right(images)
def flip_ud(images):
    return tfi.flip_up_down(images)
def contrast(images):
    return tfi.adjust_contrast(images, 3)
def ver_shift(images):
    return np.roll(images, 15, axis=1)
def bright(images):
    return tfi.adjust_brightness(images, 2)
def rotate(images, angle):
    return sp.ndimage.rotate(
        images, angle, axes=(1,2),
        reshape=False, mode='nearest')


In [ ]:
predictions = model_convTL.predict(X_val).flatten()
out = (predictions > 0.5).astype(int).flatten()
print('Accuracy without TTA:',np.mean((y_val==out)))

In [ ]:
predictions = model_convTL.predict(X_val).flatten()
predictions = np.expand_dims(predictions, axis = 0)
predictions = np.append(predictions, np.expand_dims(model_convTL.predict(flip_lr(X_val)).flatten(),axis = 0), axis=0)
#predictions = np.append(predictions, np.expand_dims(model_convTL.predict(flip_ud(X_val)).flatten(),axis=0), axis=0)
predictions = np.append(predictions, np.expand_dims(model_convTL.predict(contrast(X_val)).flatten(),axis=0), axis=0)
predictions = np.append(predictions, np.expand_dims(model_convTL.predict(rot90(X_val,1)).flatten(),axis=0), axis=0)
#predictions = np.append(predictions, np.expand_dims(model_convTL.predict(rot90(X_val,2)).flatten(),axis=0), axis=0)
predictions = np.append(predictions, np.expand_dims(model_convTL.predict(rot90(X_val,3)).flatten(),axis=0), axis=0)
#predictions = np.append(predictions, np.expand_dims(model_convTL.predict(hor_shift(X_val)).flatten(),axis=0), axis=0)
#predictions = np.append(predictions, np.expand_dims(model_convTL.predict(ver_shift(X_val)).flatten(),axis=0), axis=0)
predictions = np.append(predictions, np.expand_dims(model_convTL.predict(bright(X_val)).flatten(),axis=0), axis=0)

predictions = np.mean(predictions, axis=0)
out = (predictions > 0.5).astype(int).flatten()
print('Accuracy with TTA:',np.mean((y_val==out)))

flip_lr+rot90+rot270+contrast3 = 0.9460539460539461

flip_lr+rot90+rot270+contrast3+bright2 = 0.9470529470529471